In [1]:
import dask.dataframe as dd
import dask.array as da
import dask.bag as db
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing, impute, decomposition
from sklearn.experimental import enable_iterative_imputer

In [2]:
trainSet = dd.read_csv('../../data/train_merged_data.csv', blocksize=50e6)
trainSet

,Unnamed: 0,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,D_42,D_43,D_44,B_4,D_45,B_5,R_2,D_46,D_47,D_48,D_49,B_6,B_7,B_8,D_50,D_51,B_9,R_3,D_52,P_3,B_10,D_53,S_5,B_11,S_6,D_54,R_4,S_7,B_12,S_8,D_55,D_56,B_13,R_5,D_58,S_9,B_14,D_59,D_60,D_61,B_15,S_11,D_62,D_63,D_64,D_65,B_16,B_17,B_18,B_19,D_66,B_20,D_68,S_12,R_6,S_13,B_21,D_69,B_22,D_70,D_71,D_72,S_15,B_23,D_73,P_4,D_74,D_75,D_76,B_24,R_7,D_77,B_25,B_26,D_78,D_79,R_8,R_9,S_16,D_80,R_10,R_11,B_27,D_81,D_82,S_17,R_12,B_28,R_13,D_83,R_14,R_15,D_84,R_16,B_29,B_30,S_18,D_86,D_87,R_17,R_18,D_88,B_31,S_19,R_19,B_32,S_20,R_20,R_21,B_33,D_89,R_22,R_23,D_91,D_92,D_93,D_94,R_24,R_25,D_96,S_22,S_23,S_24,S_25,S_26,D_102,D_103,D_104,D_105,D_106,D_107,B_36,B_37,R_26,R_27,B_38,D_108,D_109,D_110,D_111,B_39,D_112,B_40,S_27,D_113,D_114,D_115,D_116,D_117,D_118,D_119,D_120,D_121,D_122,D_123,D_124,D_125,D_126,D_127,D_128,D_129,B_41,B_42,D_130,D_131,D_132,D_133,R_28,D_134,D_135,D_136,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145,target
npartitions=326,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,int64,object,object,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,object,object,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

## Limpieza

In [3]:
#trainSet = trainSet.drop(columns=['Unnamed: 0','customer_ID', 'S_2'])

Eliminamos la columna Unnamed: 0 que nos agrega dask al guardar los datos, y las columnas customer_ID y S_" (fechas)  ya que no la vamos a usar en nuestro modelo.

In [4]:
trainSet.isna().compute()

,Unnamed: 0,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,...,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145,target
0,False,False,False,False,False,False,False,False,False,False,...,True,True,False,False,False,True,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,True,True,False,False,False,True,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,True,True,False,False,False,True,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,True,True,False,False,False,True,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,True,True,False,False,False,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16970,False,False,False,False,False,False,False,False,False,False,...,True,True,False,False,False,True,False,False,False,False
16971,False,False,False,False,False,False,False,False,False,False,...,True,True,False,False,False,True,False,False,False,False
16972,False,False,False,False,False,False,False,False,False,False,...,True,True,False,False,False,True,False,False,False,False
16973,False,False,False,False,False,False,False,False,False,False,...,True,True,False,False,False,True,False,False,False,False


In [5]:
nansColumns = trainSet.columns[trainSet.isna().compute().any()].tolist()

En nansColumns obtenemos la lista de columnas que poseen valores NaN.

In [6]:
len(nansColumns)/len(trainSet.columns)

0.6354166666666666

Determinamos que el 64% de las columnas poseen valores NaN.

In [7]:
dropColumns = trainSet.columns[trainSet.isna().compute().sum()/len(trainSet) >= .2].tolist() 
dropColumns

['D_42',
 'D_43',
 'D_46',
 'D_49',
 'D_50',
 'D_53',
 'D_56',
 'S_9',
 'B_17',
 'D_66',
 'D_73',
 'D_76',
 'D_77',
 'R_9',
 'D_82',
 'B_29',
 'D_87',
 'D_88',
 'D_105',
 'D_106',
 'R_26',
 'D_108',
 'D_110',
 'D_111',
 'B_39',
 'S_27',
 'B_42',
 'D_132',
 'D_134',
 'D_135',
 'D_136',
 'D_137',
 'D_138',
 'D_142']

En dropColumns obtenemos la lista de columnas que vamos a eliminar por tener 20% o mas de datos NaN. 

In [8]:
trainSet = trainSet.drop(columns=dropColumns)
trainSet.describe()

,Unnamed: 0,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,D_44,B_4,D_45,B_5,R_2,D_47,D_48,B_6,B_7,B_8,D_51,B_9,R_3,D_52,P_3,B_10,S_5,B_11,S_6,D_54,R_4,S_7,B_12,S_8,D_55,B_13,R_5,D_58,B_14,D_59,D_60,D_61,B_15,S_11,D_62,D_65,B_16,B_18,B_19,B_20,D_68,S_12,R_6,S_13,B_21,D_69,B_22,D_70,D_71,D_72,S_15,B_23,P_4,D_74,D_75,B_24,R_7,B_25,B_26,D_78,D_79,R_8,S_16,D_80,R_10,R_11,B_27,D_81,S_17,R_12,B_28,R_13,D_83,R_14,R_15,D_84,R_16,B_30,S_18,D_86,R_17,R_18,B_31,S_19,R_19,B_32,S_20,R_20,R_21,B_33,D_89,R_22,R_23,D_91,D_92,D_93,D_94,R_24,R_25,D_96,S_22,S_23,S_24,S_25,S_26,D_102,D_103,D_104,D_107,B_36,B_37,R_27,B_38,D_109,D_112,B_40,D_113,D_114,D_115,D_116,D_117,D_118,D_119,D_120,D_121,D_122,D_123,D_124,D_125,D_126,D_127,D_128,D_129,B_41,D_130,D_131,D_133,R_28,D_139,D_140,D_141,D_143,D_144,D_145,target
npartitions=1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


Eliminamos las columnas presentes en dropColumns del dataset.

In [9]:
nansColumns = trainSet.columns[trainSet.isna().compute().any()].tolist()

In [10]:
len(nansColumns)/len(trainSet.columns) #0.6387434554973822, 0.5605095541401274

0.5569620253164557

Al volver a calcular el porcentaje de columnas que poseen valores NaN, podemos obsevar que hemos reducido la cantidad de columnas con valores NaN en un 7.8%.

In [11]:
dropRows = trainSet.columns[trainSet.isna().compute().sum()/len(trainSet) <= .05].tolist() 
dropRows

['Unnamed: 0',
 'customer_ID',
 'S_2',
 'P_2',
 'D_39',
 'B_1',
 'B_2',
 'R_1',
 'D_41',
 'B_3',
 'D_44',
 'B_4',
 'D_45',
 'B_5',
 'R_2',
 'D_47',
 'B_6',
 'B_7',
 'B_8',
 'D_51',
 'B_9',
 'R_3',
 'D_52',
 'B_10',
 'S_5',
 'B_11',
 'S_6',
 'D_54',
 'R_4',
 'B_12',
 'S_8',
 'D_55',
 'B_13',
 'R_5',
 'D_58',
 'B_14',
 'D_59',
 'D_60',
 'B_15',
 'S_11',
 'D_63',
 'D_64',
 'D_65',
 'B_16',
 'B_18',
 'B_19',
 'B_20',
 'D_68',
 'S_12',
 'R_6',
 'S_13',
 'B_21',
 'D_69',
 'B_22',
 'D_70',
 'D_71',
 'D_72',
 'S_15',
 'B_23',
 'P_4',
 'D_74',
 'D_75',
 'B_24',
 'R_7',
 'B_25',
 'B_26',
 'D_78',
 'D_79',
 'R_8',
 'S_16',
 'D_80',
 'R_10',
 'R_11',
 'B_27',
 'D_81',
 'S_17',
 'R_12',
 'B_28',
 'R_13',
 'D_83',
 'R_14',
 'R_15',
 'D_84',
 'R_16',
 'B_30',
 'S_18',
 'D_86',
 'R_17',
 'R_18',
 'B_31',
 'S_19',
 'R_19',
 'B_32',
 'S_20',
 'R_20',
 'R_21',
 'B_33',
 'D_89',
 'R_22',
 'R_23',
 'D_91',
 'D_92',
 'D_93',
 'D_94',
 'R_24',
 'R_25',
 'D_96',
 'S_22',
 'S_23',
 'S_24',
 'S_25',
 'S_26',
 '

En dropRows obtenemos las columnas que poseen 5% o menos de valores NaN, las cuales serviran como subset para eliminar las filas correspondientes. 

In [12]:
trainSet = trainSet.dropna(subset=dropRows)

In [13]:
nansColumns = trainSet.columns[trainSet.isna().compute().any()].tolist()
len(nansColumns)/len(trainSet.columns) #0.6387434554973822, 0.5605095541401274, 0.03821656050955414

0.0379746835443038

Al volver a calcular el porcentaje de columnas que poseen valores NaN, podemos obsevar que hemos reducido la cantidad de columnas con valores NaN en un 52.2%.

In [14]:
trainSet.describe()

,Unnamed: 0,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,D_44,B_4,D_45,B_5,R_2,D_47,D_48,B_6,B_7,B_8,D_51,B_9,R_3,D_52,P_3,B_10,S_5,B_11,S_6,D_54,R_4,S_7,B_12,S_8,D_55,B_13,R_5,D_58,B_14,D_59,D_60,D_61,B_15,S_11,D_62,D_65,B_16,B_18,B_19,B_20,D_68,S_12,R_6,S_13,B_21,D_69,B_22,D_70,D_71,D_72,S_15,B_23,P_4,D_74,D_75,B_24,R_7,B_25,B_26,D_78,D_79,R_8,S_16,D_80,R_10,R_11,B_27,D_81,S_17,R_12,B_28,R_13,D_83,R_14,R_15,D_84,R_16,B_30,S_18,D_86,R_17,R_18,B_31,S_19,R_19,B_32,S_20,R_20,R_21,B_33,D_89,R_22,R_23,D_91,D_92,D_93,D_94,R_24,R_25,D_96,S_22,S_23,S_24,S_25,S_26,D_102,D_103,D_104,D_107,B_36,B_37,R_27,B_38,D_109,D_112,B_40,D_113,D_114,D_115,D_116,D_117,D_118,D_119,D_120,D_121,D_122,D_123,D_124,D_125,D_126,D_127,D_128,D_129,B_41,D_130,D_131,D_133,R_28,D_139,D_140,D_141,D_143,D_144,D_145,target
npartitions=1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [15]:
trainSet = trainSet.fillna(trainSet.mean()).compute()

C:\Users\Alex\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\dask\dataframe\core.py:2133: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  meta = self._meta_nonempty.mean(axis=axis, skipna=skipna)


Con la funcion fillna imputamos con la media a las columnas que poseen x porcentaje de datos NaN, donde 5 < x < 20.

In [16]:
type(trainSet)

pandas.core.frame.DataFrame

In [17]:
trainSet = dd.from_pandas(trainSet, npartitions=327)

La funcion fillna, convirtio nuestro dataframe de dask a pandas, por lo que lo volvemos a convertir de tipo dask para que sea mas facil su manejo.

In [18]:
trainSet

,Unnamed: 0,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,D_44,B_4,D_45,B_5,R_2,D_47,D_48,B_6,B_7,B_8,D_51,B_9,R_3,D_52,P_3,B_10,S_5,B_11,S_6,D_54,R_4,S_7,B_12,S_8,D_55,B_13,R_5,D_58,B_14,D_59,D_60,D_61,B_15,S_11,D_62,D_63,D_64,D_65,B_16,B_18,B_19,B_20,D_68,S_12,R_6,S_13,B_21,D_69,B_22,D_70,D_71,D_72,S_15,B_23,P_4,D_74,D_75,B_24,R_7,B_25,B_26,D_78,D_79,R_8,S_16,D_80,R_10,R_11,B_27,D_81,S_17,R_12,B_28,R_13,D_83,R_14,R_15,D_84,R_16,B_30,S_18,D_86,R_17,R_18,B_31,S_19,R_19,B_32,S_20,R_20,R_21,B_33,D_89,R_22,R_23,D_91,D_92,D_93,D_94,R_24,R_25,D_96,S_22,S_23,S_24,S_25,S_26,D_102,D_103,D_104,D_107,B_36,B_37,R_27,B_38,D_109,D_112,B_40,D_113,D_114,D_115,D_116,D_117,D_118,D_119,D_120,D_121,D_122,D_123,D_124,D_125,D_126,D_127,D_128,D_129,B_41,D_130,D_131,D_133,R_28,D_139,D_140,D_141,D_143,D_144,D_145,target
npartitions=327,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,int64,object,object,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,object,object,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64
51,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16916,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

In [19]:
nansColumns = trainSet.columns[trainSet.isna().compute().any()].tolist()
len(nansColumns)/len(trainSet.columns) #0.6387434554973822, 0.5605095541401274, 0.03821656050955414, 0

0.0

Finalmente hemos terminado esta primera iteracion de la limpieza de los datos, al reducir la cantidad de columnas con valores NaN al 0%.

## Transformacion

## Tranformacion de valores

In [20]:
objectColumns = trainSet.loc[:, trainSet.dtypes == object]
objectColumns

,customer_ID,S_2,D_63,D_64
npartitions=327,,,,
0,object,object,object,object
51,...,...,...,...
...,...,...,...,...
16916,...,...,...,...
16998,...,...,...,...


Obtenemos las columnas cuyos valores son de tipo object.

In [21]:
objectColumns['D_63'].unique().compute()

0    CR
1    CO
2    CL
3    XM
4    XZ
5    XL
Name: D_63, dtype: object

Obtenemos la lista de valores unicos en la columna D_63.

In [22]:
objectColumns['D_64'].unique().compute()

0     O
1     U
2     R
3    -1
Name: D_64, dtype: object

Obtenemos la lista de valores unicos en la columna D_64.

In [23]:
objectColumns.groupby(['D_63']).count().sort_values(['D_64'], ascending=True).compute()

,customer_ID,S_2,D_64
D_63,,,
XL,5784,5784,5784
XM,6456,6456,6456
XZ,18504,18504,18504
CL,404672,404672,404672
CR,790928,790928,790928
CO,3599152,3599152,3599152


In [24]:
objectColumns.groupby(['D_64']).count().sort_values(['D_63'], ascending=True).compute()

,customer_ID,S_2,D_63
D_64,,,
-1,26713,26713,26713
R,753331,753331,753331
U,1365659,1365659,1365659
O,2679793,2679793,2679793


Obtenemos las listas de ocurrencia de los valores para decidir el orden en que los vamos a ponderar.

#### Tranformacion para D_63

In [25]:
trainSet['D_63'] = trainSet['D_63'].replace(['XL','XM','XZ','CL','CR','CO'], [1.,2.,3.,4.,5.,6.]) 
trainSet['D_63'] =  trainSet['D_63'].astype('float64')

#### Tranformacion para D_64

In [26]:
trainSet['D_64'] = trainSet['D_64'].replace(['-1','R','U','O'], [1.,2.,3.,4.]) 
trainSet['D_64'] =  trainSet['D_64'].astype('float64')

#### Comprobacion

In [27]:
trainSet[['D_63','D_64']]

,D_63,D_64
npartitions=327,,
0,float64,float64
51,...,...
...,...,...
16916,...,...
16998,...,...


In [28]:
trainSet[['D_63','D_64']].compute()

,D_63,D_64
0,5.0,4.0
0,6.0,3.0
0,6.0,2.0
0,6.0,4.0
0,6.0,4.0
...,...,...
16995,5.0,4.0
16996,6.0,4.0
16996,5.0,4.0
16997,5.0,4.0


### Subsetting

In [29]:
smallSet, bigSet = trainSet.random_split([0.15, 0.85], shuffle=True)

In [30]:
smallSet.describe().compute()

,Unnamed: 0,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,D_44,...,D_131,D_133,R_28,D_139,D_140,D_141,D_143,D_144,D_145,target
count,723210.000000,723210.000000,7.232100e+05,723210.000000,7.232100e+05,7.232100e+05,723210.000000,7.232100e+05,7.232100e+05,7.232100e+05,...,7.232100e+05,7.232100e+05,7.232100e+05,7.232100e+05,7.232100e+05,7.232100e+05,7.232100e+05,7.232100e+05,7.232100e+05,723210.000000
mean,8450.279382,0.666134,1.553295e-01,0.123911,6.188627e-01,7.956636e-02,0.221377,5.894312e-02,1.376124e-01,1.197822e-01,...,1.053995e-01,4.793022e-02,5.422368e-03,1.789288e-01,2.646213e-02,1.646212e-01,1.787852e-01,5.358488e-02,6.197829e-02,0.241469
std,4888.173150,0.243439,2.672923e-01,0.211541,4.037192e-01,2.279658e-01,0.171464,1.980665e-01,2.388492e-01,2.241063e-01,...,2.980090e-01,1.753728e-01,2.076900e-02,3.790577e-01,1.449644e-01,3.484818e-01,3.789338e-01,1.846512e-01,1.929373e-01,0.427974
min,0.000000,-0.442723,2.358775e-08,-0.622866,4.219620e-07,1.534223e-09,-0.487692,3.800650e-08,1.512150e-07,2.222165e-09,...,1.315117e-09,6.771411e-09,5.719286e-08,3.767347e-10,5.186710e-09,3.580881e-08,1.653580e-08,4.524265e-08,2.458379e-08,0.000000
25%,4690.500000,0.532402,5.095812e-03,0.010014,1.691590e-01,3.243059e-03,0.145253,3.180914e-03,5.867940e-03,4.280188e-03,...,3.082298e-03,3.037105e-03,2.819235e-03,3.332925e-03,2.818033e-03,3.341745e-03,3.339181e-03,3.101657e-03,3.375282e-03,0.000000
50%,8912.000000,0.744801,9.943303e-03,0.036772,8.160410e-01,6.155674e-03,0.195572,6.089911e-03,1.460498e-02,8.341804e-03,...,5.909767e-03,5.871025e-03,5.314433e-03,6.541281e-03,5.472386e-03,6.478974e-03,6.425002e-03,5.832507e-03,6.525986e-03,0.000000
75%,13048.250000,0.891238,2.795586e-01,0.182136,1.003440e+00,9.011098e-03,0.238297,8.940516e-03,2.348647e-01,1.348442e-01,...,8.655060e-03,8.508377e-03,7.798533e-03,9.511190e-03,7.933502e-03,9.611175e-03,9.545918e-03,8.661120e-03,9.525040e-03,1.000000
max,16937.000000,1.009999,4.450499e+00,1.324056,1.010000e+00,3.256284e+00,5.264421,6.168849e+00,1.571396e+00,5.625931e+00,...,1.482625e+00,1.510000e+00,1.009981e+00,1.010000e+00,1.009997e+00,1.334799e+00,1.010000e+00,1.343331e+00,4.819433e+00,1.000000


In [31]:
bigSet.describe().compute()

,Unnamed: 0,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,D_44,...,D_131,D_133,R_28,D_139,D_140,D_141,D_143,D_144,D_145,target
count,4.102286e+06,4.102286e+06,4.102286e+06,4.102286e+06,4.102286e+06,4.102286e+06,4.102286e+06,4.102286e+06,4.102286e+06,4.102286e+06,...,4.102286e+06,4.102286e+06,4.102286e+06,4.102286e+06,4.102286e+06,4.102286e+06,4.102286e+06,4.102286e+06,4.102286e+06,4.102286e+06
mean,8.460538e+03,6.659333e-01,1.542666e-01,1.239387e-01,6.186105e-01,7.957485e-02,2.215088e-01,5.855094e-02,1.372931e-01,1.195259e-01,...,1.051737e-01,4.784758e-02,5.436517e-03,1.785964e-01,2.644309e-02,1.643170e-01,1.784697e-01,5.365041e-02,6.187481e-02,2.411065e-01
std,4.885004e+03,2.435922e-01,2.660265e-01,2.120735e-01,4.037625e-01,2.283294e-01,1.718619e-01,1.964354e-01,2.384701e-01,2.232352e-01,...,2.977402e-01,1.751221e-01,2.108905e-02,3.787748e-01,1.448845e-01,3.482403e-01,3.786631e-01,1.848894e-01,1.926282e-01,4.277549e-01
min,0.000000e+00,-4.589548e-01,5.026190e-09,-3.571172e+00,9.192280e-09,2.058457e-09,-6.271320e-01,3.440668e-09,6.285293e-09,5.153088e-10,...,5.173169e-10,1.243154e-09,1.097922e-09,5.286876e-10,3.725073e-09,1.650100e-10,5.549692e-09,2.500991e-09,1.226024e-09,0.000000e+00
25%,4.440750e+03,5.229786e-01,4.872267e-03,9.814641e-03,1.422739e-01,3.023264e-03,1.412527e-01,3.033707e-03,5.752119e-03,4.073280e-03,...,2.918082e-03,2.863779e-03,2.620278e-03,3.191740e-03,2.656949e-03,3.224685e-03,3.161726e-03,2.882389e-03,3.201467e-03,0.000000e+00
50%,8.737000e+03,7.431604e-01,9.687735e-03,3.552502e-02,8.154650e-01,5.976190e-03,1.810249e-01,5.947973e-03,1.219159e-02,8.261381e-03,...,5.787699e-03,5.673627e-03,5.164681e-03,6.348351e-03,5.211209e-03,6.332683e-03,6.271192e-03,5.693046e-03,6.281043e-03,0.000000e+00
75%,1.285175e+04,8.869783e-01,2.723525e-01,1.664453e-01,1.003283e+00,8.917830e-03,2.293165e-01,8.883649e-03,2.103310e-01,1.341189e-01,...,8.580670e-03,8.389659e-03,7.627959e-03,9.400097e-03,7.782608e-03,9.438746e-03,9.379237e-03,8.488385e-03,9.409571e-03,1.000000e+00
max,1.694000e+04,1.010000e+00,5.355393e+00,1.324060e+00,1.010000e+00,3.006102e+00,5.482888e+00,8.988807e+00,1.625262e+00,5.634724e+00,...,1.491624e+00,1.509999e+00,1.009996e+00,1.010000e+00,1.010000e+00,1.339910e+00,1.010000e+00,1.343331e+00,4.827630e+00,1.000000e+00


In [32]:
smallSet

,Unnamed: 0,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,D_44,B_4,D_45,B_5,R_2,D_47,D_48,B_6,B_7,B_8,D_51,B_9,R_3,D_52,P_3,B_10,S_5,B_11,S_6,D_54,R_4,S_7,B_12,S_8,D_55,B_13,R_5,D_58,B_14,D_59,D_60,D_61,B_15,S_11,D_62,D_63,D_64,D_65,B_16,B_18,B_19,B_20,D_68,S_12,R_6,S_13,B_21,D_69,B_22,D_70,D_71,D_72,S_15,B_23,P_4,D_74,D_75,B_24,R_7,B_25,B_26,D_78,D_79,R_8,S_16,D_80,R_10,R_11,B_27,D_81,S_17,R_12,B_28,R_13,D_83,R_14,R_15,D_84,R_16,B_30,S_18,D_86,R_17,R_18,B_31,S_19,R_19,B_32,S_20,R_20,R_21,B_33,D_89,R_22,R_23,D_91,D_92,D_93,D_94,R_24,R_25,D_96,S_22,S_23,S_24,S_25,S_26,D_102,D_103,D_104,D_107,B_36,B_37,R_27,B_38,D_109,D_112,B_40,D_113,D_114,D_115,D_116,D_117,D_118,D_119,D_120,D_121,D_122,D_123,D_124,D_125,D_126,D_127,D_128,D_129,B_41,D_130,D_131,D_133,R_28,D_139,D_140,D_141,D_143,D_144,D_145,target
npartitions=327,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,int64,object,object,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64
51,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16916,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [33]:
bigSet

,Unnamed: 0,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,D_44,B_4,D_45,B_5,R_2,D_47,D_48,B_6,B_7,B_8,D_51,B_9,R_3,D_52,P_3,B_10,S_5,B_11,S_6,D_54,R_4,S_7,B_12,S_8,D_55,B_13,R_5,D_58,B_14,D_59,D_60,D_61,B_15,S_11,D_62,D_63,D_64,D_65,B_16,B_18,B_19,B_20,D_68,S_12,R_6,S_13,B_21,D_69,B_22,D_70,D_71,D_72,S_15,B_23,P_4,D_74,D_75,B_24,R_7,B_25,B_26,D_78,D_79,R_8,S_16,D_80,R_10,R_11,B_27,D_81,S_17,R_12,B_28,R_13,D_83,R_14,R_15,D_84,R_16,B_30,S_18,D_86,R_17,R_18,B_31,S_19,R_19,B_32,S_20,R_20,R_21,B_33,D_89,R_22,R_23,D_91,D_92,D_93,D_94,R_24,R_25,D_96,S_22,S_23,S_24,S_25,S_26,D_102,D_103,D_104,D_107,B_36,B_37,R_27,B_38,D_109,D_112,B_40,D_113,D_114,D_115,D_116,D_117,D_118,D_119,D_120,D_121,D_122,D_123,D_124,D_125,D_126,D_127,D_128,D_129,B_41,D_130,D_131,D_133,R_28,D_139,D_140,D_141,D_143,D_144,D_145,target
npartitions=327,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,int64,object,object,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64
51,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16916,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [34]:
smallSet.to_csv('../../data/train_merged_clean_small_data.csv', single_file=True)
bigSet.to_csv('../../data/train_merged_clean_big_data.csv', single_file=True)

['c:\\Users\\Alex\\Desktop\\IA_Avanzada\\data\\train_merged_clean_big_data.csv']

In [35]:
trainSmallSet = pd.read_csv('../../data/train_merged_clean_small_data.csv')
trainSmallSet

,Unnamed: 0,Unnamed: 0.1,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,...,D_131,D_133,R_28,D_139,D_140,D_141,D_143,D_144,D_145,target
0,23,9358,8cff42b70b8c710e494ad76a6a2b81ce8fed1c43a0c207...,2017-09-02,0.996272,0.000498,0.033549,1.006211,0.002662,0.099488,...,0.001268,0.006127,0.009415,0.000679,0.001851,0.002921,0.002069,0.006225,0.009271,0
1,24,13414,ca4f526810d21c72d7a355a21946d7c46277b046cd8fb7...,2017-09-11,0.993062,0.003368,0.007620,0.810955,0.005453,0.151026,...,0.003427,0.008814,0.002297,0.002566,0.009391,0.006044,0.002108,0.008260,0.000254,0
2,24,7907,770148eb1c05ed2bc015cb46633eef6a9ced0444a2cebf...,2017-06-29,0.397454,0.004070,0.842780,0.021770,0.508095,0.170206,...,0.004853,0.006172,0.009537,1.007677,1.007559,0.901299,1.005034,0.006123,0.551438,1
3,39,11466,ac80171e4c62c8acd8f08d9e4c6b1787b9671910048156...,2017-07-01,0.571684,0.007852,0.115360,0.024719,0.008315,0.060586,...,0.007114,0.000299,0.002745,1.008829,0.005483,0.972598,1.007914,0.557387,0.372534,0
4,13,8884,85f497102552c783dd0cf4482267aec8c1e7f4c6c884b4...,2018-03-18,0.483349,0.214539,0.278715,0.128913,0.002420,0.979543,...,0.007534,0.006594,0.000280,1.005153,0.008178,0.867849,1.002246,0.006808,0.094289,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
723205,16958,6671,651a73ec9deb94fb6bcf2c5e882e1ca9a6c1db4fe52012...,2017-12-26,0.893274,0.005365,0.095702,0.075081,0.025859,0.067235,...,0.003917,0.002726,0.002592,0.008956,0.006085,0.005904,0.006083,0.000763,0.002611,0
723206,16988,12518,bd05fe55a6efe09903ad678f98ef2253668347a510a2a9...,2017-12-09,0.627439,0.000067,0.117831,0.329922,0.009249,0.111463,...,0.001808,0.002716,0.007284,0.007989,0.003593,0.001812,0.004497,0.009677,0.009686,0
723207,16946,4533,44b7d6fa5f14f1c1c7360b7825f1f15b5d5b7ece27efa6...,2017-07-22,0.451686,0.240364,0.043077,0.813675,0.004497,0.339234,...,0.005507,0.005911,0.007616,0.003416,0.001337,0.005753,0.005904,0.004389,0.007512,0
723208,16962,12183,b8393593caaf84e12ace8ec42dd43a7260a92de219407a...,2017-04-21,0.929417,0.004392,0.006815,0.811576,0.007347,0.120580,...,0.005344,0.007095,0.009300,0.009470,0.000312,0.006554,0.000144,0.001093,0.005243,0


Obtención de las columnas importantes

In [36]:
columns_trainSmallSet  = trainSmallSet [['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_68']]
labelEncoder = preprocessing.LabelEncoder()
columns_trainSmallSet  = columns_trainSmallSet .apply(labelEncoder.fit_transform)
columns_trainSmallSet 

,B_30,B_38,D_114,D_116,D_117,D_120,D_126,D_63,D_64,D_68
0,0,1,1,0,4,0,2,5,3,6
1,0,1,1,0,5,0,2,4,3,6
2,1,4,0,0,0,0,1,5,1,2
3,1,2,1,0,3,0,2,5,3,3
4,0,2,1,0,4,1,2,3,3,6
...,...,...,...,...,...,...,...,...,...,...
723205,0,4,1,0,4,0,2,5,3,5
723206,0,4,1,0,4,0,1,5,3,5
723207,0,0,1,0,0,0,2,5,3,3
723208,0,1,1,0,5,0,1,3,3,6


Imputación de datos por medio de OneHotEncoding para los datos no numéricos

In [37]:
mat = preprocessing.OneHotEncoder()
mat.fit(columns_trainSmallSet )
one_hot_labels = mat.transform(columns_trainSmallSet ).toarray()
one_hot_labels

array([[1., 0., 0., ..., 0., 0., 1.],
       [1., 0., 0., ..., 0., 0., 1.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 1.],
       [1., 0., 0., ..., 0., 0., 1.]])

In [38]:
cat_col_names = mat.get_feature_names_out(list(columns_trainSmallSet .columns))
cat_col_names

array(['B_30_0', 'B_30_1', 'B_30_2', 'B_38_0', 'B_38_1', 'B_38_2',
       'B_38_3', 'B_38_4', 'B_38_5', 'B_38_6', 'D_114_0', 'D_114_1',
       'D_116_0', 'D_116_1', 'D_117_0', 'D_117_1', 'D_117_2', 'D_117_3',
       'D_117_4', 'D_117_5', 'D_117_6', 'D_120_0', 'D_120_1', 'D_126_0',
       'D_126_1', 'D_126_2', 'D_63_0', 'D_63_1', 'D_63_2', 'D_63_3',
       'D_63_4', 'D_63_5', 'D_64_0', 'D_64_1', 'D_64_2', 'D_64_3',
       'D_68_0', 'D_68_1', 'D_68_2', 'D_68_3', 'D_68_4', 'D_68_5',
       'D_68_6'], dtype=object)

In [39]:
cat_col_df = pd.DataFrame(one_hot_labels, columns = cat_col_names)
cat_col_df

,B_30_0,B_30_1,B_30_2,B_38_0,B_38_1,B_38_2,B_38_3,B_38_4,B_38_5,B_38_6,...,D_64_1,D_64_2,D_64_3,D_68_0,D_68_1,D_68_2,D_68_3,D_68_4,D_68_5,D_68_6
0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
723205,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
723206,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
723207,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
723208,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [40]:
trainSmallSet   = trainSmallSet.drop(columns = columns_trainSmallSet .columns)
trainSmallSet  = trainSmallSet.join(cat_col_df)

In [41]:
trainSmallSet_x = trainSmallSet.drop(columns=['target'])
trainSmallSet_y = trainSmallSet[['target']]

Imputación por PCA para el dataset a sacar

In [42]:
trainSmallSet = trainSmallSet.sort_values(by=['customer_ID'])
pca_trainSmallSet_x = trainSmallSet_x.drop(columns=['customer_ID', 'S_2'])
imp = impute.IterativeImputer()
imputed_trainSmallSet_x = imp.fit_transform(pca_trainSmallSet_x)

In [43]:
imp_trainSmallSet_x = pd.DataFrame(imputed_trainSmallSet_x, columns = list(pca_trainSmallSet_x.columns))

In [59]:
pca = decomposition.PCA(n_components=20)
trainSmallSet_x_imputed_pca_df = pca.fit_transform(imp_trainSmallSet_x)

Renombramiento de las columnas en una denominación más general

In [60]:
trainSmallSet_x_imputed_pca_df = pd.DataFrame(trainSmallSet_x_imputed_pca_df, columns = [f'column_{i}' for i in range(trainSmallSet_x_imputed_pca_df.shape[1])])
trainSmallSet_x_imputed_pca_df

,column_0,column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8,column_9,column_10,column_11,column_12,column_13,column_14,column_15,column_16,column_17,column_18,column_19
0,8453.217259,-986.438274,-0.237870,-0.202708,-0.112456,0.118246,-0.098874,-0.707770,1.395742,1.065074,-0.240863,-0.227747,0.283484,0.160946,0.303235,0.013126,0.710233,0.216523,0.139706,1.034024
1,9349.369229,2969.323176,-0.230675,-0.163126,-0.100315,0.099923,-0.123072,-0.771184,1.722902,-0.546371,-0.748690,0.339288,0.813922,0.104702,-0.696493,-0.817992,-0.617044,-0.076586,0.190070,0.263378
2,8131.302292,-2401.278465,-0.209537,0.088882,-0.097364,-0.236784,-0.069353,1.068889,-2.803240,-0.414667,0.537566,0.311564,-0.711785,-0.410864,-0.404752,-0.223932,0.432039,0.043180,1.135434,0.410832
3,8903.871967,1072.889365,-0.215740,0.024188,-0.091726,-0.229790,-0.071551,0.611268,-1.587590,0.160667,-0.399192,0.353155,0.801767,-0.274666,-1.106406,0.164557,1.495387,-0.017101,0.916341,0.063774
4,8358.127788,-1450.910011,-0.216081,-0.003469,-0.093240,-0.203762,-0.044805,0.512470,-1.337662,0.208273,-0.480587,0.303576,0.771871,-0.200190,-0.398479,-0.293642,-0.264843,-1.063941,1.016208,-0.246069
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
723205,-8656.658276,138.885311,-0.220998,-0.039983,-0.154049,-0.190505,-0.071293,0.170082,-0.587139,0.294516,-0.462120,0.081805,0.790563,-0.146094,-0.534530,-0.205200,0.017247,0.849735,-0.017811,-0.236734
723206,-7392.645323,5847.701322,-0.218911,-0.081199,-0.135350,-0.089870,-0.016126,0.276532,-0.777934,0.420846,-0.383138,0.358334,0.316371,-0.165526,-0.886757,0.099443,0.533440,0.999068,-0.425417,0.428868
723207,-9117.849467,-1948.814544,-0.221199,-0.089933,-0.166114,-0.182032,-0.105925,-0.103715,0.110900,-0.235452,-0.194969,-0.217376,0.486869,-0.020690,-1.311269,1.351396,-0.198932,0.316870,-0.744425,-0.406394
723208,-7441.386339,5515.247849,-0.230879,-0.061708,-0.150176,-0.190361,-0.093085,-0.616051,1.338900,-0.214500,-0.752563,0.077605,0.773806,0.088196,-0.554948,-0.361047,-0.847473,0.027967,0.244455,0.403171


In [61]:
trainSmallSet_x

,Unnamed: 0,Unnamed: 0.1,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,...,D_64_1,D_64_2,D_64_3,D_68_0,D_68_1,D_68_2,D_68_3,D_68_4,D_68_5,D_68_6
0,23,9358,8cff42b70b8c710e494ad76a6a2b81ce8fed1c43a0c207...,2017-09-02,0.996272,0.000498,0.033549,1.006211,0.002662,0.099488,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,24,13414,ca4f526810d21c72d7a355a21946d7c46277b046cd8fb7...,2017-09-11,0.993062,0.003368,0.007620,0.810955,0.005453,0.151026,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,24,7907,770148eb1c05ed2bc015cb46633eef6a9ced0444a2cebf...,2017-06-29,0.397454,0.004070,0.842780,0.021770,0.508095,0.170206,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,39,11466,ac80171e4c62c8acd8f08d9e4c6b1787b9671910048156...,2017-07-01,0.571684,0.007852,0.115360,0.024719,0.008315,0.060586,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,13,8884,85f497102552c783dd0cf4482267aec8c1e7f4c6c884b4...,2018-03-18,0.483349,0.214539,0.278715,0.128913,0.002420,0.979543,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
723205,16958,6671,651a73ec9deb94fb6bcf2c5e882e1ca9a6c1db4fe52012...,2017-12-26,0.893274,0.005365,0.095702,0.075081,0.025859,0.067235,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
723206,16988,12518,bd05fe55a6efe09903ad678f98ef2253668347a510a2a9...,2017-12-09,0.627439,0.000067,0.117831,0.329922,0.009249,0.111463,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
723207,16946,4533,44b7d6fa5f14f1c1c7360b7825f1f15b5d5b7ece27efa6...,2017-07-22,0.451686,0.240364,0.043077,0.813675,0.004497,0.339234,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
723208,16962,12183,b8393593caaf84e12ace8ec42dd43a7260a92de219407a...,2017-04-21,0.929417,0.004392,0.006815,0.811576,0.007347,0.120580,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [62]:
trainSmallSet_pca = trainSmallSet_y.join(trainSmallSet_x[['customer_ID','S_2']]).join(trainSmallSet_x_imputed_pca_df)
trainSmallSet_pca

,target,customer_ID,S_2,column_0,column_1,column_2,column_3,column_4,column_5,column_6,...,column_10,column_11,column_12,column_13,column_14,column_15,column_16,column_17,column_18,column_19
0,0,8cff42b70b8c710e494ad76a6a2b81ce8fed1c43a0c207...,2017-09-02,8453.217259,-986.438274,-0.237870,-0.202708,-0.112456,0.118246,-0.098874,...,-0.240863,-0.227747,0.283484,0.160946,0.303235,0.013126,0.710233,0.216523,0.139706,1.034024
1,0,ca4f526810d21c72d7a355a21946d7c46277b046cd8fb7...,2017-09-11,9349.369229,2969.323176,-0.230675,-0.163126,-0.100315,0.099923,-0.123072,...,-0.748690,0.339288,0.813922,0.104702,-0.696493,-0.817992,-0.617044,-0.076586,0.190070,0.263378
2,1,770148eb1c05ed2bc015cb46633eef6a9ced0444a2cebf...,2017-06-29,8131.302292,-2401.278465,-0.209537,0.088882,-0.097364,-0.236784,-0.069353,...,0.537566,0.311564,-0.711785,-0.410864,-0.404752,-0.223932,0.432039,0.043180,1.135434,0.410832
3,0,ac80171e4c62c8acd8f08d9e4c6b1787b9671910048156...,2017-07-01,8903.871967,1072.889365,-0.215740,0.024188,-0.091726,-0.229790,-0.071551,...,-0.399192,0.353155,0.801767,-0.274666,-1.106406,0.164557,1.495387,-0.017101,0.916341,0.063774
4,0,85f497102552c783dd0cf4482267aec8c1e7f4c6c884b4...,2018-03-18,8358.127788,-1450.910011,-0.216081,-0.003469,-0.093240,-0.203762,-0.044805,...,-0.480587,0.303576,0.771871,-0.200190,-0.398479,-0.293642,-0.264843,-1.063941,1.016208,-0.246069
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
723205,0,651a73ec9deb94fb6bcf2c5e882e1ca9a6c1db4fe52012...,2017-12-26,-8656.658276,138.885311,-0.220998,-0.039983,-0.154049,-0.190505,-0.071293,...,-0.462120,0.081805,0.790563,-0.146094,-0.534530,-0.205200,0.017247,0.849735,-0.017811,-0.236734
723206,0,bd05fe55a6efe09903ad678f98ef2253668347a510a2a9...,2017-12-09,-7392.645323,5847.701322,-0.218911,-0.081199,-0.135350,-0.089870,-0.016126,...,-0.383138,0.358334,0.316371,-0.165526,-0.886757,0.099443,0.533440,0.999068,-0.425417,0.428868
723207,0,44b7d6fa5f14f1c1c7360b7825f1f15b5d5b7ece27efa6...,2017-07-22,-9117.849467,-1948.814544,-0.221199,-0.089933,-0.166114,-0.182032,-0.105925,...,-0.194969,-0.217376,0.486869,-0.020690,-1.311269,1.351396,-0.198932,0.316870,-0.744425,-0.406394
723208,0,b8393593caaf84e12ace8ec42dd43a7260a92de219407a...,2017-04-21,-7441.386339,5515.247849,-0.230879,-0.061708,-0.150176,-0.190361,-0.093085,...,-0.752563,0.077605,0.773806,0.088196,-0.554948,-0.361047,-0.847473,0.027967,0.244455,0.403171


Agrupamiento de los usuarios en el dataset final, de tal forma que se tenga un cliente por fila del archivo

In [63]:
trainSmallSet = trainSmallSet_pca.groupby('customer_ID', as_index = False).mean()

In [64]:
trainSmallSet

,customer_ID,target,column_0,column_1,column_2,column_3,column_4,column_5,column_6,column_7,...,column_10,column_11,column_12,column_13,column_14,column_15,column_16,column_17,column_18,column_19
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.0,6397.501907,-10104.584324,-0.230231,-0.121854,-0.124036,0.129043,-0.097399,-0.653849,...,-0.619522,-0.186811,0.813343,0.108428,-0.986798,-0.898190,-0.524100,-0.023766,0.135741,0.269928
1,00000fd6641609c6ece5454664794f0340ad84dddce9a2...,0.0,6391.218184,-10094.614182,-0.229584,-0.203984,-0.129463,0.100098,-0.141379,-0.671607,...,0.004898,-0.531235,0.243852,0.152303,0.588966,0.100074,0.096238,0.001139,-0.008701,0.334406
2,00001b22f846c82c51f6e3958ccd81970162bae8b007e8...,0.0,6377.896696,-10073.477481,-0.233251,-0.182213,-0.137842,0.093621,-0.161345,-0.681673,...,0.133387,-0.174952,-0.305611,0.082972,0.853723,0.698838,-0.350248,-0.037446,-0.142460,-0.756442
3,000041bdba6ecadd89a52d11886e8eaaec9325906c9723...,0.0,6376.388602,-10071.084648,-0.230502,0.097643,-0.133321,0.091601,-0.122672,-0.428077,...,0.009457,-0.518589,0.119190,0.101003,0.039341,0.126869,0.838566,0.264819,-0.671916,0.383505
4,00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8a...,0.0,6358.291484,-10042.370639,-0.227853,-0.087171,-0.132026,-0.117923,-0.144135,-0.388776,...,-0.692867,0.175252,0.916309,0.038638,-0.928099,1.161622,-0.602860,0.047381,-0.050382,0.018224
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
342157,ffff39cc22a375d07369980d02d617883dd28ad81a6aa3...,0.0,-6351.576297,10053.604827,-0.232590,-0.130624,-0.140117,-0.077628,-0.118294,-0.205806,...,0.270655,0.015576,-0.630385,-0.088906,-0.958784,0.544814,-0.218002,0.699878,-0.234193,-0.351888
342158,ffff41c8a52833b56430603969b9ca48d208e7c192c6a4...,0.0,-6362.132950,10070.354666,-0.236058,-0.173083,-0.132455,0.213168,-0.062451,-0.436055,...,0.625980,-0.125316,-1.255722,-0.001726,-0.764720,-0.356212,-0.944084,0.206589,0.718443,1.099131
342159,ffff518bb2075e4816ee3fe9f3b152c57fc0e6f01bf7fd...,0.0,-6371.181511,10084.711669,-0.218254,0.048744,-0.128785,-0.216440,-0.069347,0.397034,...,-0.737237,0.470286,1.232281,-0.179983,-0.053452,0.126542,-0.781202,0.535958,0.386159,-0.504580
342160,ffffa5c46bc8de74f5a4554e74e239c8dee6b9baf38814...,1.0,-6393.802908,10120.604180,-0.221367,0.069901,-0.129571,-0.192446,-0.082371,0.281224,...,0.371408,-0.207015,-0.467500,-0.195958,-1.193190,0.253868,0.599899,-0.404865,0.431735,-0.874429


In [65]:
trainSmallSet.to_csv('../../data/train_user_collapsed_small_data.csv')